In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
from tqdm.notebook import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


How many events are at each creepmeter?

In [2]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.1990,5 m,NaN,30,Balikburnu
1,PARK,C461,C461_10T,2004-09-29T02:20:00.000000Z,2022-06-20T12:40:00.000000Z,0.001667,10T,35.7240,-120.2820,NaN,NaN,30,Highway 46
2,PARK,C462,C462_1T,2021-08-19T19:10:11.000000Z,2024-03-22T01:05:11.000000Z,0.016667,1T,35.7240,-120.2820,NaN,NaN,30,Highway 46
3,UTA,CAN1,CAN1_30T,2012-06-12T15:33:00.000000Z,2017-06-15T08:33:00.000000Z,0.000556,30T,38.0198,-110.0374,7.5 m,NaN,45,Canyonlands
4,NAF,CER1,CER1_30S,2019-10-09T06:46:00.000000Z,2024-05-23T10:25:30.000000Z,0.033333,30S,40.8947,32.7773,20 m,NaN,29,Cerkes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,HOL,XSJ3,XSJ3_10T,2007-02-15T20:19:59.000000Z,2016-12-08T18:09:59.000000Z,0.001667,10T,36.8360,-121.5210,20 m,NaN,30,San Juan Bautista #3
121,PARK,XTA1,XTA1_10T,1985-12-18T00:00:00.000000Z,2024-10-09T18:50:00.000000Z,0.001667,10T,35.8900,-120.4270,30 m,1 m,30,Taylor Ranch
122,PARK,XVA1,XVA1_10T,1987-05-29T00:00:00.000000Z,2017-10-23T22:10:00.000000Z,0.001667,10T,35.9220,-120.4620,30 m,1 m,30,Varian
123,EAF,YZE1,YZE1_1T,2023-09-07T10:36:07.000000Z,2024-05-26T12:59:07.000000Z,0.016667,1T,38.1791,38.7526,20 m,NaN,30,Yazica East


In [3]:
Columns = ['Network','Creepmeter_full_name','Creepmeter_abbrv','Latitude','Longitude',\
           'Sampling rate, mins','Event_id','ST','ET','Displacement, mm','Duration, hrs','File_code']
df_all = pd.DataFrame(columns=Columns)
e_id = 0
for i in tqdm(range(len(Creepmeter_dataframe))):
    #e_id = 0
    
    event_slip = []  
    event_duration = []
    event_code = []
    print(Creepmeter_dataframe["File_code"].iloc[i])
    try:
        df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".\
            format(q =Creepmeter_dataframe.File_code.iloc[i]),index_col=0)
        df_picks['Longitude'] = [Creepmeter_dataframe['Longitude'].iloc[i]]*len(df_picks)
        df_picks['Latitude'] = [Creepmeter_dataframe['Latitude'].iloc[i]]*len(df_picks)
        df_picks['Network'] = [Creepmeter_dataframe['Network'].iloc[i]]*len(df_picks)
        df_picks['Creepmeter_full_name'] = [Creepmeter_dataframe['Creepmeter_full_name'].iloc[i]]*len(df_picks)
        df_picks['Creepmeter_abbrv'] = [Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]]*len(df_picks)
        df_picks['Sampling rate, mins'] = [Creepmeter_dataframe['Sampling rate, mins'].iloc[i]]*len(df_picks)
        df_picks['File_code'] = [Creepmeter_dataframe['File_code'].iloc[i]]*len(df_picks)
        
        
        path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q=Creepmeter_dataframe.File_code.iloc[i])
        st = obspy.read(path)
        tr = st[0]
        tr.interpolate(sampling_rate=tr.stats.sampling_rate)
        tm = tr.times("utcdatetime")
        creep = tr.data
        sample_rate =  round(tr.stats.delta)/60
        tm_dt = []
        for w in range(len(tm)):
            tm_new = tm[w].datetime
            tm_dt.append(tm_new)

        sample_rate = Creepmeter_dataframe['Sampling rate, mins'].iloc[i]
        tm_int, creep_int, upsampled = cep.interpolate_linear(tm_dt,creep,sample_rate)

        if 'Quality_check_new' in df_picks.columns:
            print(Creepmeter_dataframe["File_code"].iloc[i], 'finsihed qc')
            df_picks.drop(df_picks[df_picks['Quality_check_new'] != 'CE'].index,inplace=True)
            df_picks.reset_index(inplace=True,drop=True)
            print(len(df_picks))
            for j in tqdm(range(len(df_picks))):

                #print(j,df_picks.ST_new.iloc[j],df_picks.ET_new.iloc[j])
                #print('new QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST_new.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET_new.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                #print(event_creep)
                e_id +=1
                event_code.append("{:04d}".format(e_id) + '_{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST_new'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET_new'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])

        elif 'Quality_check' in df_picks.columns:
            print(Creepmeter_dataframe["File_code"].iloc[i],'old QC')
            df_picks.drop(df_picks[df_picks['Quality_check'] != 'CE'].index,inplace=True)
            df_picks.reset_index(inplace=True,drop=True)
            print(len(df_picks))
            for j in range(len(df_picks)):
                print(j,df_picks.ST_new.iloc[j],df_picks.ET_new.iloc[j])
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
        
        else:
            print(len(df_picks))
            print(Creepmeter_dataframe["File_code"].iloc[i],'no QC')
            for j in range(len(df_picks)):
                
                #print('old QC')
                boolarr = np.logical_and(np.array(tm_int)>=np.datetime64(df_picks.ST.iloc[j]),\
                                        np.array(tm_int)<=np.datetime64(df_picks.ET.iloc[j]))
                event_creep = creep_int[boolarr]
                event_time = tm_int[boolarr]
                e_id +=1
                event_code.append('{k}'.format(k=Creepmeter_dataframe.Creepmeter_abbrv.iloc[i]) + "{:04d}".format(e_id))
                event_slip.append(event_creep[-1] - event_creep[0])
                event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
            df_picks['Displacement, mm'] = event_slip
            df_picks['Duration, hrs'] = event_duration
            df_picks['Event_id'] = event_code
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
            df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                dt.round(Creepmeter_dataframe['Sampling rate, mins'].iloc[i])
        

        df_all = pd.concat([df_all,df_picks[Columns]],ignore_index=True)
        #df_picks.to_csv("../../../Data/{p}/Picks/{q}_picks_details.csv".format(p=Creepmeter_dataframe.Network.iloc[i],q =Creepmeter_dataframe.File_code.iloc[i]))
    except FileNotFoundError:
        print('no picks at {k}'.format(k=Creepmeter_dataframe["File_code"].iloc[i]))
    


  0%|          | 0/125 [00:00<?, ?it/s]

BAL1_1T
7
BAL1_1T no QC
C461_10T


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_83835/2337362041.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all,df_picks[Columns]],ignore_index=True)


61
C461_10T no QC
C462_1T
17
C462_1T no QC
CAN1_30T
21
CAN1_30T no QC
CER1_30S
21
CER1_30S no QC
CER1_1T
11
CER1_1T no QC
CFW1_1T
CFW1_1T finsihed qc
44


  0%|          | 0/44 [00:00<?, ?it/s]

...

XTA1_10T
XTA1_10T finsihed qc
204


  0%|          | 0/204 [00:00<?, ?it/s]

XVA1_10T
XVA1_10T finsihed qc
319


  0%|          | 0/319 [00:00<?, ?it/s]

YZE1_1T
3
YZE1_1T no QC
YZW1_1T
11
YZW1_1T no QC


In [5]:
#df_all.dropna(inplace=True)
df_all.sort_values(by='ST',inplace=True)
df_all.reset_index(inplace=True,drop=True)
df_all

,Network,Creepmeter_full_name,Creepmeter_abbrv,Latitude,Longitude,"Sampling rate, mins",Event_id,ST,ET,"Displacement, mm","Duration, hrs",File_code
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,4731_XMR1,1980-02-19 15:20:00,1980-02-20 21:20:00,0.163750,30.000000,XMR1_10T
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,4732_XMR1,1980-02-21 06:20:00,1980-02-28 15:20:00,0.567109,177.000000,XMR1_10T
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,4733_XMR1,1980-07-22 17:40:00,1980-08-04 08:50:00,0.761968,303.166667,XMR1_10T
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,4734_XMR1,1980-09-10 10:10:00,1980-09-16 22:50:00,0.441859,156.666667,XMR1_10T
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,4735_XMR1,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127001,8.500000,XMR1_10T
...,...,...,...,...,...,...,...,...,...,...,...,...
6034,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,0181_CFW1,2024-02-03 17:03:00,2024-02-04 04:34:00,0.021122,11.516667,CFW1_1T
6035,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,0182_CFW1,2024-02-04 09:44:00,2024-02-05 14:04:00,0.088898,28.333333,CFW1_1T
6036,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,1468_FCR1,2024-02-12 15:16:00,2024-02-15 21:57:00,0.073456,78.683333,FCR1_1T
6037,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,1469_FCR1,2024-02-16 06:05:00,2024-02-16 08:26:00,0.225962,2.350000,FCR1_1T


In [6]:
'''file_codes = Creepmeter_dataframe["File_code"]
sub_region = Creepmeter_dataframe["Sub_region_map"]

subregion_events = []
for i in range(len(df_all)):
    alpha = df_all["File_code"].iloc[i]
    beta = np.where(file_codes==alpha)
    subregion_events.append(sub_region[beta[0][0]])
df_all['sub_region_maps'] = subregion_events'''

'file_codes = Creepmeter_dataframe["File_code"]\nsub_region = Creepmeter_dataframe["Sub_region_map"]\n\nsubregion_events = []\nfor i in range(len(df_all)):\n    alpha = df_all["File_code"].iloc[i]\n    beta = np.where(file_codes==alpha)\n    subregion_events.append(sub_region[beta[0][0]])\ndf_all[\'sub_region_maps\'] = subregion_events'

In [7]:
print(np.unique(df_all['Creepmeter_abbrv']))

['BAL1' 'C461' 'C462' 'CAN1' 'CER1' 'CFW1' 'CHA1' 'CHE1' 'COZ1' 'CPP1'
 'CRR1' 'CTM1' 'CWC3' 'CWN1' 'DS30' 'DU30' 'ESZ1' 'FCR1' 'FE30' 'GD'
 'GOK1' 'GOZ1' 'HAM1' 'HAS1' 'HAT1' 'INW1' 'IR30' 'ISW1' 'KAR1' 'KIR1'
 'NS45' 'ORM1' 'PAN1' 'PAS1' 'PET1' 'QE30' 'QW30' 'RA' 'RB' 'RR45' 'SEL1'
 'SH30' 'SIV1' 'SJN1' 'SU30' 'TAS1' 'TEP1' 'WKR1' 'WSZ1' 'X461' 'XGH1'
 'XHR1' 'XHR2' 'XHR3' 'XHSW' 'XMD1' 'XMM1' 'XMR1' 'XPK1' 'XPK2' 'XRSW'
 'XSC1' 'XSJ2' 'XSJ3' 'XTA1' 'XVA1' 'YZE1' 'YZW1']


In [8]:
event_id = []
for i in range(len(df_all)):
    event_id.append("{:04d}".format(i) + '_{k}'.format(k=df_all.Creepmeter_abbrv.iloc[i]))
df_all['Event_id'] = event_id
df_all

,Network,Creepmeter_full_name,Creepmeter_abbrv,Latitude,Longitude,"Sampling rate, mins",Event_id,ST,ET,"Displacement, mm","Duration, hrs",File_code
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0000_XMR1,1980-02-19 15:20:00,1980-02-20 21:20:00,0.163750,30.000000,XMR1_10T
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0001_XMR1,1980-02-21 06:20:00,1980-02-28 15:20:00,0.567109,177.000000,XMR1_10T
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0002_XMR1,1980-07-22 17:40:00,1980-08-04 08:50:00,0.761968,303.166667,XMR1_10T
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0003_XMR1,1980-09-10 10:10:00,1980-09-16 22:50:00,0.441859,156.666667,XMR1_10T
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0004_XMR1,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127001,8.500000,XMR1_10T
...,...,...,...,...,...,...,...,...,...,...,...,...
6034,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6034_CFW1,2024-02-03 17:03:00,2024-02-04 04:34:00,0.021122,11.516667,CFW1_1T
6035,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6035_CFW1,2024-02-04 09:44:00,2024-02-05 14:04:00,0.088898,28.333333,CFW1_1T
6036,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6036_FCR1,2024-02-12 15:16:00,2024-02-15 21:57:00,0.073456,78.683333,FCR1_1T
6037,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6037_FCR1,2024-02-16 06:05:00,2024-02-16 08:26:00,0.225962,2.350000,FCR1_1T


In [9]:
#df_all['magnitude'] = np.log10(df_all['Displacement, mm'])
#df_all['Date'] = df_all['ST']
df_all

,Network,Creepmeter_full_name,Creepmeter_abbrv,Latitude,Longitude,"Sampling rate, mins",Event_id,ST,ET,"Displacement, mm","Duration, hrs",File_code
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0000_XMR1,1980-02-19 15:20:00,1980-02-20 21:20:00,0.163750,30.000000,XMR1_10T
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0001_XMR1,1980-02-21 06:20:00,1980-02-28 15:20:00,0.567109,177.000000,XMR1_10T
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0002_XMR1,1980-07-22 17:40:00,1980-08-04 08:50:00,0.761968,303.166667,XMR1_10T
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0003_XMR1,1980-09-10 10:10:00,1980-09-16 22:50:00,0.441859,156.666667,XMR1_10T
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0004_XMR1,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127001,8.500000,XMR1_10T
...,...,...,...,...,...,...,...,...,...,...,...,...
6034,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6034_CFW1,2024-02-03 17:03:00,2024-02-04 04:34:00,0.021122,11.516667,CFW1_1T
6035,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6035_CFW1,2024-02-04 09:44:00,2024-02-05 14:04:00,0.088898,28.333333,CFW1_1T
6036,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6036_FCR1,2024-02-12 15:16:00,2024-02-15 21:57:00,0.073456,78.683333,FCR1_1T
6037,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6037_FCR1,2024-02-16 06:05:00,2024-02-16 08:26:00,0.225962,2.350000,FCR1_1T


In [10]:
df_all.sort_values(by='ST')

,Network,Creepmeter_full_name,Creepmeter_abbrv,Latitude,Longitude,"Sampling rate, mins",Event_id,ST,ET,"Displacement, mm","Duration, hrs",File_code
0,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0000_XMR1,1980-02-19 15:20:00,1980-02-20 21:20:00,0.163750,30.000000,XMR1_10T
1,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0001_XMR1,1980-02-21 06:20:00,1980-02-28 15:20:00,0.567109,177.000000,XMR1_10T
2,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0002_XMR1,1980-07-22 17:40:00,1980-08-04 08:50:00,0.761968,303.166667,XMR1_10T
3,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0003_XMR1,1980-09-10 10:10:00,1980-09-16 22:50:00,0.441859,156.666667,XMR1_10T
4,HOL,Melendy Ranch,XMR1,36.5950,-121.1870,10T,0004_XMR1,1980-12-13 12:20:00,1980-12-13 20:50:00,0.127001,8.500000,XMR1_10T
...,...,...,...,...,...,...,...,...,...,...,...,...
6034,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6034_CFW1,2024-02-03 17:03:00,2024-02-04 04:34:00,0.021122,11.516667,CFW1_1T
6035,HAY,Fremont Winery,CFW1,37.5320,-121.9520,1T,6035_CFW1,2024-02-04 09:44:00,2024-02-05 14:04:00,0.088898,28.333333,CFW1_1T
6036,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6036_FCR1,2024-02-12 15:16:00,2024-02-15 21:57:00,0.073456,78.683333,FCR1_1T
6037,HOL,Fox Creek,FCR1,36.6925,-121.3061,1T,6037_FCR1,2024-02-16 06:05:00,2024-02-16 08:26:00,0.225962,2.350000,FCR1_1T


In [11]:
df_all.to_csv("../../Data/DATA_tidied/Picks/All_picks_05_MAR_2025.csv")

In [26]:
df_all['disp'] = df_all['Displacement, mm']

'''Columns = ['Network','Creepmeter_full_name','Creepmeter_abbrv','Latitude','Longitude',\
           'Sampling rate, mins','Event_id','ST','ET','Displacement, mm','Duration, hrs','Quality_check','File_code']
df_all = pd.DataFrame(columns=Columns)
for r in tqdm(range(len(all_creepmeters))):
    creepmeter = all_creepmeters[r]
    creepmeter_info = Creepmeter_dataframe.loc[Creepmeter_dataframe.index[Creepmeter_dataframe.Creepmeter_abbrv==creepmeter]]

    e_id = 0
    for i in range(len(creepmeter_info)): 
        event_slip = []  
        event_duration = []
        event_code = []
        try:
            print("../../../Data/{p}/Picks/{q}_picks.csv".format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i]))
            df_picks = pd.read_csv("../../../Data/{p}/Picks/{q}_picks.csv".\
                                   format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i]),index_col=0)
            '''try:
                checker = df_picks['Quality_check_new']
                is_it_there=True
            except IndexError:
                is_it_there=False'''
            is_it_there=True
            if is_it_there == True:
                #df_picks.drop(df_picks[df_picks['Quality_check']!='CE'].index,inplace=True)
                df_picks.reset_index(inplace=True)
                df_picks['Longitude'] = [creepmeter_info['Longitude'].iloc[i]]*len(df_picks)
                df_picks['Latitude'] = [creepmeter_info['Latitude'].iloc[i]]*len(df_picks)
                df_picks['Network'] = [creepmeter_info['Network'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter_full_name'] = [creepmeter_info['Creepmeter_full_name'].iloc[i]]*len(df_picks)
                df_picks['Creepmeter_abbrv'] = [creepmeter_info['Creepmeter_abbrv'].iloc[i]]*len(df_picks)
                df_picks['Sampling rate, mins'] = [creepmeter_info['Sampling rate, mins'].iloc[i]]*len(df_picks)
                df_picks['File_code'] = [creepmeter_info['File_code'].iloc[i]]*len(df_picks)
                try:
                    df_picks.drop(['index'],axis=1,inplace=True)
                except:
                    dummy=10
                try:
                    df_picks.drop(['level_0'],axis=1,inplace=True)
                except:
                    dummy=10
                
                path = "../../../Data/{p}/SAC/{q}.SAC".format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i])
                st = obspy.read(path)
                tr = st[0]
                tr.interpolate(sampling_rate=tr.stats.sampling_rate)
                tm = tr.times("utcdatetime")
                creep = tr.data
                sample_rate =  round(tr.stats.delta)/60
                tm_dt = []
                for w in range(len(tm)):
                    tm_new = tm[w].datetime
                    tm_dt.append(tm_new)

                tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate,5)

                
                for j in range(len(df_picks)):
                    boolarr = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_picks.ST_new.iloc[j]).replace(tzinfo=None),\
                                            np.array(tm_int)<=pd.to_datetime(df_picks.ET_new.iloc[j]).replace(tzinfo=None))
                    event_creep = creep_int[boolarr]
                    event_time = tm_int[boolarr]
                    try:
                        if df_picks['Quality_check_new'].iloc[j] == 'CE':
                            e_id +=1
                            event_code.append('{k}_'.format(k=creepmeter_info.Creepmeter_abbrv.iloc[0]) + "{:04d}".format(e_id))
                            event_slip.append(event_creep[-1] - event_creep[0])
                            event_duration.append((pd.to_datetime(event_time[-1]) - pd.to_datetime(event_time[0]))/dt.timedelta(hours=1))
                        else:
                            event_code.append('None Event')
                            event_slip.append(np.nan)
                            event_duration.append(np.nan)
                    except IndexError:
                        dummy=10 
                df_picks['Displacement, mm'] = event_slip
                df_picks['Duration, hrs'] = event_duration
                df_picks['Event_id'] = event_code
                df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
                df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
                df_picks['ST'] = pd.Series(pd.to_datetime(df_picks['ST'])).\
                    dt.round("{k}min".format(k=creepmeter_info['Sampling rate, mins'].iloc[i]))
                df_picks['ET'] = pd.Series(pd.to_datetime(df_picks['ET'])).\
                    dt.round("{k}min".format(k=creepmeter_info['Sampling rate, mins'].iloc[i]))
                
                

                df_all = pd.concat([df_all,df_picks[Columns]],ignore_index=True)
                df_picks.to_csv("../../../Data/{p}/Picks/{q}_picks_details.csv".format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i]))
                #df_disp['{k}'.format(k=Creepmeter_dataframe["File_code"].iloc[r])] = df_picks['Displacement, mm'].dropna()

        except:
            dummy=10
        #df_all.to_csv("../../../Data/all_creep_event_picks.csv")
        #df_disp.to_csv("../../../Data/all_creep_event_displacements.csv")
        #np.save("../../../Data/all_creep_event_displacements.npy", df_disp) 
'''

In [ ]:
np.unique(df_all["Creepmeter_abbrv"])

In [ ]:
Count = [1]*len(df_all)
print(len(Count))
df_all["count"] = Count
df_all

In [ ]:
df_stats = df_all.drop(['Network','Creepmeter_full_name','Latitude','Longitude',\
           'Sampling rate, mins','Event_id','ST','ET','Quality_check'],axis=1)



#groupby(['Creepmeter_full_name']).sum()

In [ ]:


grouped = df_stats.groupby(['File_code']).sum()
grouped

In [ ]:
grouped["count"].loc['Carr Ranch']

In [ ]:
Hayward = []
Parkfield = []
SanJuanBautista = []
SouthernCali = []
Turkey = []
Pakistan = []

In [3]:
import numpy as np
np.arange(1000)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18